In [1]:
import os

import numpy as np
from PIL import Image

from cleverhans.attacks import FastGradientMethod

import tensorflow as tf
from tensorflow.contrib.slim.nets import inception

slim = tf.contrib.slim

In [2]:
tf.flags.DEFINE_string(
    'master', '', 'The address of the TensorFlow master to use.')

tf.flags.DEFINE_string(
    'checkpoint_path', '', 'Path to checkpoint for inception network.')

tf.flags.DEFINE_string(
    'input_dir', '', 'Input directory with images.')

tf.flags.DEFINE_string(
    'output_dir', '', 'Output directory with images.')

tf.flags.DEFINE_float(
    'max_epsilon', 16.0, 'Maximum size of adversarial perturbation.')

tf.flags.DEFINE_integer(
    'image_width', 299, 'Width of each input images.')

tf.flags.DEFINE_integer(
    'image_height', 299, 'Height of each input images.')

tf.flags.DEFINE_integer(
    'batch_size', 16, 'How many images process at one time.')

FLAGS = tf.flags.FLAGS

In [3]:
class InceptionModel(object):
  """Model class for CleverHans library."""

  def __init__(self, num_classes):
    self.num_classes = num_classes
    self.built = False

  def __call__(self, x_input):
    """Constructs model and return probabilities for given input."""
    reuse = True if self.built else None
    with slim.arg_scope(inception.inception_v3_arg_scope()):
      _, end_points = inception.inception_v3(
          x_input, num_classes=self.num_classes, is_training=False,
          reuse=reuse)
    self.built = True
    output = end_points['Predictions']
    # Strip off the extra reshape op at the output
    probs = output.op.inputs[0]
    return probs

In [4]:
eps = 2.0 * FLAGS.max_epsilon / 255.0
batch_shape = [FLAGS.batch_size, FLAGS.image_height, FLAGS.image_width, 3]
num_classes = 1001
tf.logging.set_verbosity(tf.logging.INFO)

In [5]:
model = InceptionModel(num_classes)
fgsm = FastGradientMethod(model)

cleverhans/attacks.py:36: UserWarning: CleverHans support for supplying a callable instead of an instance of the cleverhans.model.Model class is deprecated and will be dropped on 2018-01-11.
  warnings.warn("CleverHans support for supplying a callable"


In [11]:
with tf.Graph().as_default():
    x_input = tf.placeholder(tf.float32, shape=batch_shape)
    model = inception.inception_v3(x_input)

In [12]:
model

(<tf.Tensor 'InceptionV3/Logits/SpatialSqueeze:0' shape=(16, 1000) dtype=float32>,
 {'AuxLogits': <tf.Tensor 'InceptionV3/AuxLogits/SpatialSqueeze:0' shape=(16, 1000) dtype=float32>,
  'Conv2d_1a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_1a_3x3/Relu:0' shape=(16, 149, 149, 32) dtype=float32>,
  'Conv2d_2a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_2a_3x3/Relu:0' shape=(16, 147, 147, 32) dtype=float32>,
  'Conv2d_2b_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_2b_3x3/Relu:0' shape=(16, 147, 147, 64) dtype=float32>,
  'Conv2d_3b_1x1': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_3b_1x1/Relu:0' shape=(16, 73, 73, 80) dtype=float32>,
  'Conv2d_4a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_4a_3x3/Relu:0' shape=(16, 71, 71, 192) dtype=float32>,
  'Logits': <tf.Tensor 'InceptionV3/Logits/SpatialSqueeze:0' shape=(16, 1000) dtype=float32>,
  'MaxPool_3a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/MaxPool_3a_3x3/MaxPool:0' shape=(16, 73, 73, 64) dtype=float32>,
  'MaxPool_5a_3